<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introductions" data-toc-modified-id="Introductions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introductions</a></span><ul class="toc-item"><li><span><a href="#Classifier-three-components:" data-toc-modified-id="Classifier-three-components:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Classifier three components:</a></span></li><li><span><a href="#Extension-of-the-3-components-when-applying-DNN-or-CNN:" data-toc-modified-id="Extension-of-the-3-components-when-applying-DNN-or-CNN:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Extension of the 3 components when applying DNN or CNN:</a></span></li></ul></li><li><span><a href="#Visualizing-the-loss-function" data-toc-modified-id="Visualizing-the-loss-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Visualizing the loss function</a></span></li><li><span><a href="#Optimization" data-toc-modified-id="Optimization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimization</a></span><ul class="toc-item"><li><span><a href="#Strategy-#1:-A-first-very-bad-idea-solution:-Random-search-(15.5%,-kiding??)" data-toc-modified-id="Strategy-#1:-A-first-very-bad-idea-solution:-Random-search-(15.5%,-kiding??)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Strategy #1: A first very bad idea solution: Random search (15.5%, kiding??)</a></span></li><li><span><a href="#Strategy-#2:-Random-Local-Search-(21.4%)" data-toc-modified-id="Strategy-#2:-Random-Local-Search-(21.4%)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Strategy #2: Random Local Search (21.4%)</a></span></li><li><span><a href="#Strategy-#3:-Following-the-Gradient" data-toc-modified-id="Strategy-#3:-Following-the-Gradient-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Strategy #3: Following the Gradient</a></span></li></ul></li><li><span><a href="#Computing-the-gradient" data-toc-modified-id="Computing-the-gradient-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Computing the gradient</a></span><ul class="toc-item"><li><span><a href="#Computing-the-gradient-numerically-with-finite-differences" data-toc-modified-id="Computing-the-gradient-numerically-with-finite-differences-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Computing the gradient numerically with finite differences</a></span></li><li><span><a href="#Computing-the-gradient-analytically-with-Calculus" data-toc-modified-id="Computing-the-gradient-analytically-with-Calculus-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Computing the gradient analytically with Calculus</a></span></li></ul></li><li><span><a href="#Gradient-Descent" data-toc-modified-id="Gradient-Descent-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Gradient Descent</a></span><ul class="toc-item"><li><span><a href="#Mini-batch-gradient-descent." data-toc-modified-id="Mini-batch-gradient-descent.-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Mini-batch gradient descent.</a></span></li></ul></li></ul></div>

# [Optimization: Stochastic Gradient Descent](http://cs231n.github.io/optimization-1/)



## Introductions

### Classifier three components:
* [<span class="burk">forward pass</span>] A (parameterized) **<span class="girk">score function</span>** mapping the raw image pixels to class scores (e.g. a linear function)
* <span class="burk">[backward pass</span>] A **<span class="girk">loss function</span>** that measured the quality of a particular set of parameters based on how well the induced scores agreed with the ground truth labels in the training data. We saw that there are many ways and versions of this (e.g. Softmax/SVM).
* **<span class="girk">Optimization</span>** is the process of finding the set of parameters $W$ that minimize the loss function.

### Extension of the 3 components when applying DNN or CNN:
* we will <span class="girk">revisit the first component (the parameterized function mapping)</span> and extend it to functions much more complicated than a linear mapping: First entire Neural Networks, and then Convolutional Neural Networks
* The loss functions and the optimization process will remain relatively unchanged.

## Visualizing the loss function
* 该节介绍的 对 loss function visualizing 的方法比较有趣, 以后可以借鉴.
* 通过设置 random parameters (weights), 然后再 random 方向上改变 parameter 可以改变 weights, 画出 loss function energy, 对样本平均之后可以看出 loss function 的形状.
* 具体内容看网页内容.

## Optimization
* The optimization tools used in this section is only suitable for SVM (and other convex loss function), not suitable for NN. 


### Strategy #1: A first very bad idea solution: Random search (15.5%, kiding??)

In [ ]:
# assume X_train is the data where each column is an example (e.g. 3073 x 50,000)
# assume Y_train are the labels (e.g. 1D array of 50,000)
# assume the function L evaluates the loss function

bestloss = float("inf") # Python assigns the highest possible float value
for num in xrange(1000):
  W = np.random.randn(10, 3073) * 0.0001 # generate random parameters
  loss = L(X_train, Y_train, W) # get the loss over the entire training set
  if loss < bestloss: # keep track of the best solution
    bestloss = loss
    bestW = W
  print 'in attempt %d the loss was %f, best %f' % (num, loss, bestloss)

# prints:
# in attempt 0 the loss was 9.401632, best 9.401632
# in attempt 1 the loss was 8.959668, best 8.959668
# in attempt 2 the loss was 9.044034, best 8.959668
# in attempt 3 the loss was 9.278948, best 8.959668
# in attempt 4 the loss was 8.857370, best 8.857370
# in attempt 5 the loss was 8.943151, best 8.857370
# in attempt 6 the loss was 8.605604, best 8.605604
# ... (trunctated: continues for 1000 lines)


### Strategy #2: Random Local Search (21.4%)

In [ ]:
W = np.random.randn(10, 3073) * 0.001 # generate random starting W
bestloss = float("inf")
for i in xrange(1000):
  step_size = 0.0001
  Wtry = W + np.random.randn(10, 3073) * step_size
  loss = L(Xtr_cols, Ytr, Wtry)
  if loss < bestloss:
    W = Wtry
    bestloss = loss
  print 'iter %d loss is %f' % (i, bestloss)

### Strategy #3: Following the Gradient

## Computing the gradient
* A <span class="girk">slow, approximate</span> but <span class="girk">easy</span> way (numerical gradient)
* a <span class="girk">fast</span>, exact but <span class="girk">more error-prone way</span> that <span class="girk">requires calculus</span> (analytic gradient)

### Computing the gradient numerically with finite differences

In [ ]:
def eval_numerical_gradient(f, x):
  """ 
  a naive implementation of numerical gradient of f at x 
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """ 

  fx = f(x) # evaluate function value at original point
  grad = np.zeros(x.shape)
  h = 0.00001

  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    old_value = x[ix]
    x[ix] = old_value + h # increment by h
    fxh = f(x) # evalute f(x + h)
    x[ix] = old_value # restore to previous value (very important!)

    # compute the partial derivative
    grad[ix] = (fxh - fx) / h # the slope
    it.iternext() # step to next dimension

  return grad

##### Practical considerations. 
Additionally, in practice it often works better to compute the numeric gradient using the **centered difference formula**: $[f(x+h) - f(x-h)] / 2 h$.

In [ ]:
# to use the generic code above we want a function that takes a single argument
# (the weights in our case) so we close over X_train and Y_train
def CIFAR10_loss_fun(W):
  return L(X_train, Y_train, W)

W = np.random.rand(10, 3073) * 0.001 # random weight vector
df = eval_numerical_gradient(CIFAR10_loss_fun, W) # get the gradient

The gradient tells us the slope of the loss function along every dimension, which we can use to make an update:

这里 上下 cell 之间存在调用关系

##### Update in negative gradient direction.

In [ ]:
loss_original = CIFAR10_loss_fun(W) # the original loss
print 'original loss: %f' % (loss_original, )

# lets see the effect of multiple step sizes
for step_size_log in [-10, -9, -8, -7, -6, -5,-4,-3,-2,-1]:
  step_size = 10 ** step_size_log
  W_new = W - step_size * df # new position in the weight space
  loss_new = CIFAR10_loss_fun(W_new)
  print 'for step size %f new loss: %f' % (step_size, loss_new)

# prints:
# original loss: 2.200718
# for step size 1.000000e-10 new loss: 2.200652
# for step size 1.000000e-09 new loss: 2.200057
# for step size 1.000000e-08 new loss: 2.194116
# for step size 1.000000e-07 new loss: 2.135493
# for step size 1.000000e-06 new loss: 1.647802
# for step size 1.000000e-05 new loss: 2.844355
# for step size 1.000000e-04 new loss: 25.558142
# for step size 1.000000e-03 new loss: 254.086573
# for step size 1.000000e-02 new loss: 2539.370888
# for step size 1.000000e-01 new loss: 25392.214036

##### Effect of step size.
从上面 code 运行结果可以看出, steps size 很重要, 不要太大也不要太小.
* choosing the step size (also called the learning rate) will become one of the most important (and most headache-inducing) hyperparameter settings in training a neural network.
* If we shuffle our feet carefully we can expect to make consistent but very small progress (this corresponds to having a small step size). 对于非凸问题, 走不出local-minum.
* Conversely, we can choose to make a large, confident step in an attempt to descend faster, but this may not pay off. As you can see in the code example above, at some point taking a bigger step gives a higher loss as we “overstep”.

##### A problem of efficiency.
* we had 30730 parameters in total and therefore had to perform 30,731 evaluations of the loss function to evaluate the gradient and to perform only a single parameter update. 
* This problem only gets worse, since modern Neural Networks can easily have tens of millions of parameters. Clearly, this strategy is not scalable and we need something better.

### Computing the gradient analytically with Calculus
* The numerical gradient is very simple to compute using the finite difference approximation, 
    * but **<span class="burk">the downside</span>** is that it is approximate (since we have to pick a small value of $h$, 
    * while the true gradient is defined as the limit as $h$ goes to zero), and that it is very computationally expensive to compute.
    
    
    
* The 2nd way to compute the gradient is <span class="girk">analytically using Calculus</span>, which allows us to derive a direct formula for the gradient (no approximations) that is also very fast to compute. 
    * unlike the numerical gradient it can be <span class="girk">more error prone to implement</span>, which is why in practice it is very common to <span class="girk">compute the analytic gradient</span> and <span class="girk">compare it to the numerical gradient</span> to check the correctness of your implementation
    * This is called a <span class="girk">gradient check</span>.

## Gradient Descent


In [ ]:
# Vanilla Gradient Descent
while True:
  weights_grad = evaluate_gradient(loss_fun, data, weights)
  weights += - step_size * weights_grad # perform parameter update

* This simple loop is at the core of all Neural Network libraries.
* There are other ways of performing the optimization (e.g. LBFGS), but Gradient Descent is currently by far the most common and established way of optimizing Neural Network loss functions.


### Mini-batch gradient descent.
* In large-scale applications (such as the ILSVRC challenge), the training data can have <span class="girk">on order of millions of examples</span>. Hence, it seems <span class="girk">wasteful to compute</span> the full loss function over the <span class="girk">entire training se</span>t in order to <span class="girk">perform only a single parameter update</span>


* A very common approach to addressing this challenge is to compute the gradient over <span class="girk">batches</span> of the training data.
    * For example, in current state of the art <span class="girk">ConvNets</span>, a typical batch contains <span class="girk">256 examples</span> from the entire training set of 1.2 million. 
    * This batch is then used to perform a parameter update:

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
  data_batch = sample_training_data(data, 256) # sample 256 examples
  weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
  weights += - step_size * weights_grad # perform parameter update

* 物理上说明 batch 合理性: ILSVRC 1.2 million images, 
    * only 1000 unique images (one for each class, or in other words 1200 identical copies of each image).
    * compute for all 1200 identical copies would all be the same, and when we average the data loss over all 1.2 million images we would get the exact same loss as if we only evaluated on a small subset of 1000.
    

* Extreme cases: mini-batch contains only a single example
    * This process is called **Stochastic Gradient Descent (SGD) ** (or also sometimes **on-line gradient descent**). 
    * it is relatively <span class="girk">less common</span> to see because in practice due to <span class="girk">vectorized code optimizations</span> it can be computationally much more efficient to evaluate the <span class="girk">gradient for 100 examples</span>, than <span class="girk">the gradient for one example 100 times</span>.
    
* 关于称呼: Even though <span class="girk">SGD</span> technically refers to <span class="girk">using a single example at a time</span> to evaluate the gradient, you will hear people use the term SGD even when referring to <span class="girk">mini-batch gradient descent</span> (i.e. mentions of <span class="girk">MGD</span> for “<span class="girk">Minibatch Gradient Descent</span>”, or <span class="girk">BGD</span> for “<span class="girk">Batch gradient descent</span>” are rare to see).

* The size of the mini-batch is a <span class="girk">hyperparameter</span> but it is <span class="girk">not very common</span> to <span class="girk">cross-validate</span> it.
    * It is usually based on memory constraints (if any), or set to some value, e.g. <span class="girk">32, 64 or 128</span>. 
    * We use <span class="girk">powers of 2</span> in practice because many vectorized operation implementations <span class="girk">work faster</span> when their inputs are sized in powers of 2.

## Summary
![](http://cs231n.github.io/assets/dataflow.jpeg)
Figure. Score function vs. Loss function in the pipelines.

* step size (or the learning rate) that must be set just right
* computing the numerical and analytic gradient. gradient check
* Gradient Descent. Mini-batch gradient descent.